# Sample Code

In [26]:
print('Hello World')

import os
os.getcwd()

Hello World


'/Users/wei/Desktop/recommendation-sample-main'

## 載入資料

In [27]:
import pandas as pd
import numpy as np

In [28]:
## Sample Data => 商品的基本資訊
### asin: 商品編號
### title: 商品名稱

metadata = pd.DataFrame({
    'asin': ['A1', 'A2', 'AB3', 'B4', 'B5'],
    'title': ['Hello World', 'Hello Python', 'Python Tutorial', 'Java Tutorial', 'C++ Tutorial']
})
metadata


,asin,title
0,A1,Hello World
1,A2,Hello Python
2,AB3,Python Tutorial
3,B4,Java Tutorial
4,B5,C++ Tutorial


In [29]:
## 評分資訊
### asin: 商品編號
### reviewerID: 使用者名稱
### overall: 使用者評分
### unixReviewTime/DATE: 評分時間/日期

ratings = pd.DataFrame([
    {'asin': 'A1','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'AB3','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'B4','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'AB3','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'A1','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'A2','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'B5','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'AB3','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
])
ratings


,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,AB3,USER1,5,1424304000,2015-02-19
2,B4,USER2,5,1424304000,2015-02-19
3,AB3,USER2,5,1424304000,2015-02-19
4,A1,USER1,5,1537315200,2018-09-19
5,A2,USER1,5,1537315200,2018-09-19
6,B5,USER2,5,1537315200,2018-09-19
7,AB3,USER2,5,1537315200,2018-09-19


In [30]:
metadata.head()

,asin,title
0,A1,Hello World
1,A2,Hello Python
2,AB3,Python Tutorial
3,B4,Java Tutorial
4,B5,C++ Tutorial


In [31]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,AB3,USER1,5,1424304000,2015-02-19
2,B4,USER2,5,1424304000,2015-02-19
3,AB3,USER2,5,1424304000,2015-02-19
4,A1,USER1,5,1537315200,2018-09-19


## 資料整理

In [32]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,AB3,USER1,5,1424304000,2015-02-19
2,B4,USER2,5,1424304000,2015-02-19
3,AB3,USER2,5,1424304000,2015-02-19
4,A1,USER1,5,1537315200,2018-09-19
5,A2,USER1,5,1537315200,2018-09-19
6,B5,USER2,5,1537315200,2018-09-19
7,AB3,USER2,5,1537315200,2018-09-19


## 資料切分

In [33]:
# 2018-09-01 以前的資料 => 作為推薦的參考依據（來源資料）
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
# 2018-09-01 - 2018-09-30 真實發生的資料 => 作為驗證結果使用（測試資料）
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
# 把測試資料整理成每個使用者分別買了什麼的格式
# 實際上買了哪些東西的真實資料 => 需要拿來跟預測結果做比較的
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
# 把測試資料整理出有哪些使用者 => 知道有哪些使用者需要被推薦的
users = list(ratings_testings_by_user.keys())


In [34]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,AB3,USER1,5,1424304000,2015-02-19
2,B4,USER2,5,1424304000,2015-02-19
3,AB3,USER2,5,1424304000,2015-02-19


In [35]:
ratings_testings

,asin,reviewerID,overall,unixReviewTime,DATE
4,A1,USER1,5,1537315200,2018-09-19
5,A2,USER1,5,1537315200,2018-09-19
6,B5,USER2,5,1537315200,2018-09-19
7,AB3,USER2,5,1537315200,2018-09-19


In [36]:
# 知道有哪些使用者需要被推薦的
users

['USER1', 'USER2']

In [37]:
# 實際上買了哪些東西的真實資料 => 需要拿來跟預測結果做比較的
ratings_testings_by_user

{'USER1': ['A1', 'A2'], 'USER2': ['B5', 'AB3']}

## 產生推薦

In [38]:
def recommender(training_data, users=[], k=2):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code: 如何將 training_data （過去的資料）轉成 recommendations（推薦結果 = 每個使用者分別推薦哪些商品的集合）
    '''
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{}

## 結果評估

In [39]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user: # 真實被購買的商品資料中
        if d in ratings_by_user: # 有哪些是有被推薦到的
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.0

In [40]:
ratings_testings_by_user # 真實被購買的商品資料

{'USER1': ['A1', 'A2'], 'USER2': ['B5', 'AB3']}

In [41]:
ratings_by_user # 利用訓練資料學習的推薦商品

{}